In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Lambda
from keras.models import Sequential
from moviepy.editor import VideoFileClip
from IPython.display import display, HTML

Using TensorFlow backend.


In [2]:
def load_data(data_folder, correction):

    data = pd.read_csv(os.path.join(data_folder, "driving_log.csv"))

    # Get the file paths from csv and extract the filename only
    X = data[['center', 'left', 'right']].values
    # Get the steering values and adjust them for the left and right images
    y = data['steering'].values

    # Get indices of training and validation data
    train_ind, valid_ind = train_test_split(range(X.shape[0]), test_size=0.2, random_state=42)

    # For training use center, left, and right images
    X_train = np.array([fname.replace("\\", "/").split("/")[-1] for fname in X[train_ind].flatten(order="F")])
    y_train = np.hstack((y[train_ind], y[train_ind]+correction, y[train_ind]-correction))
    # For validation only use center images
    X_valid = np.array([fname.replace("\\", "/").split("/")[-1] for fname in X[valid_ind][:,0].flatten(order="F")])
    y_valid = y[valid_ind]

    return X_train, X_valid, y_train, y_valid

In [3]:
def read_img(file_name):
    # Read in image as BGR format
    img = cv2.imread(file_name)
    return img

In [4]:
def flip_img(img, steering):
    # Flip image horizontally and adjust steering angle
    flip = cv2.flip(img, 1)
    angle = -steering
    return flip, angle

In [10]:
def data_generator(img_folder, X_data, y_data, batch_size=128, is_training=True):
    num_samples = len(X_data)
    while 1:
        X_data, y_data = shuffle(X_data, y_data)
        for offset in range(0, num_samples, batch_size):
            X_batch = X_data[offset:offset+batch_size]
            y_batch = y_data[offset:offset+batch_size]

            images = []
            angles = []

            for fname, angle in zip(X_batch, y_batch):
                img = read_img(os.path.join(img_folder, fname))
                ang = angle
                if is_training and np.random.rand() < 0.5:
                    img, ang = flip_img(img, angle)
                img = preprocess_img(img)
                images.append(img)
                angles.append(ang)

            X = np.array(images)
            y = np.array(angles)

            yield shuffle(X, y)

In [6]:
def preprocess_img(img):
    # Remove the sky and the car
    crop = img[65:-25, :, :] 
    print(type(crop))
    # Resize to fit for Nvidia model and change color space accordingly
    resize = cv2.resize(crop, (200, 66))
    color = cv2.cvtColor(resize, cv2.COLOR_BGR2YUV)
    return color

In [15]:
def build_model():
    # Build the Nvidia model with small adaptions
    model = Sequential()
    model.add(Lambda(lambda x: x/255.0-0.5, input_shape=(66,200,3)))
    model.add(Conv2D(24, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))

    return model

In [29]:
def train_model(model, X_train, X_valid, y_train, y_valid, batch_size, epochs, img_folder):
    # Create callbacks for saving checkpoints and early stopping
    checkpoint = ModelCheckpoint("model_test-{epoch:02d}.h5", save_best_only=True)
    early_stopping = EarlyStopping(min_delta=0.1, patience=3)

    model.compile(loss='mse', optimizer="adam")
    # Number of batches to yield from generator for one epoch
    train_steps = (X_train.shape[0]//3//batch_size)*batch_size
    valid_steps = (X_valid.shape[0]//batch_size)*batch_size
    print(train_steps, valid_steps)
    # Train the model
    model.fit_generator(data_generator(img_folder, X_train, y_train, batch_size, True), train_steps, max_q_size=1, 
                      nb_epoch=epochs, validation_data=data_generator(img_folder, X_valid, y_valid, batch_size, False),
                      nb_val_samples=valid_steps, callbacks=[checkpoint, early_stopping])

In [11]:
data_folder = os.path.join(os.path.curdir, "data")
img_folder = os.path.join(data_folder, "IMG")
batch_size = 128
learning_rate = 0.001
correction = 0.2
epochs = 3

In [16]:
X_train, X_valid, y_train, y_valid = load_data(data_folder, correction)
model = build_model()

In [10]:
X_train = X_train[:255*3]
X_valid = X_valid[:256]
y_train = y_train[:255*3]
y_valid = y_valid[:256]

In [109]:
train_model(model, X_train, X_valid, y_train, y_valid, batch_size, epochs, img_folder)

128 256
Epoch 1/3
128/128 [==============================] - 4s - loss: 0.0201 - val_loss: 0.0143
Epoch 2/3
128/128 [==============================] - 2s - loss: 0.0110 - val_loss: 0.0174
Epoch 3/3
128/128 [==============================] - 2s - loss: 0.0174 - val_loss: 0.0135


In [2]:
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip = VideoFileClip("video.mp4").subclip(16.0,21.0)
clip.write_gif("result.gif",fps=25)


[MoviePy] Building file result.gif with imageio


 99%|█████████▉| 125/126 [00:07<00:00, 16.95it/s]
